In [1]:
import gc
gc.collect()

0

In [2]:
import numpy as np
import random 
import tensorflow as tf
import datetime


In [3]:
text= open('55998-0.txt',encoding= 'UTF-8',).read()
print('text length in number of charcater', len(text))

print('head of text file:')
print (text[:1000])

text length in number of charcater 2891848
head of text file:
﻿The Project Gutenberg EBook of London Labour and the London Poor (Vol. 1
of 4), by Henry Mayhew

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
www.gutenberg.org.  If you are not located in the United States, you'll have
to check the laws of the country where you are located before using this ebook.



Title: London Labour and the London Poor (Vol. 1 of 4)

Author: Henry Mayhew

Release Date: November 19, 2017 [EBook #55998]

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK LONDON LABOUR ***




Produced by Henry Flower, Jonathan Ingram, Suzanne Lybarger,
the booksmiths at eBookForge and the Online Distributed
Proofreading Team at http://www.pgdp.

In [4]:
chars=sorted(list(set(text)))
char_size=len(chars)
print('number of chars:', char_size)
print(chars)
print('Done')

number of chars: 116
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '£', '·', 'á', 'â', 'æ', 'ç', 'é', 'ë', 'œ', 'Δ', 'α', 'ε', 'λ', 'μ', 'ο', 'π', 'υ', 'ί', 'ῳ', '‘', '’', '“', '”', '„', '☞', '\ufeff']
Done


In [5]:
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))
#print(ix_to_char)
#print (chars_to_ix)

In [6]:
def sample(prediction):
    r = random.uniform(0,1)
    s = 0
    char_id = len(prediction) - 1
    for i in range(len(prediction)):
        s += prediction[i]
        if s >= r:
            char_id = i
            break
    char_one_hot = np.zeros(shape=[char_size])
    char_one_hot[char_id] = 1.0
    return char_one_hot
    

In [7]:
len_per_section = 15
skip = 2

sections = []
next_chars = []
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])

#Vectorize input and output
X = np.zeros((len(sections), len_per_section, char_size))
y = np.zeros((len(sections), char_size))
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1

In [8]:
batch_size = 512
max_steps = 80000
log_every = 3000
test_every = 6000
hidden_nodes = 1024
test_start = 'I am thinking that'
checkpoint_directory = 'ckpt'

#Create a checkpoint directory
if tf.gfile.Exists(checkpoint_directory):
    tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))

training data size: 1445917
approximate steps per epoch: 2824


In [9]:
graph = tf.Graph()
with graph.as_default():
    ###########
    #Prep
    ###########
    #Variables and placeholders
    global_step = tf.Variable(0)
    
    data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
    labels = tf.placeholder(tf.float32, [batch_size, char_size])
    
    #Prep LSTM Operation
    #Input gate: weights for input, weights for previous output, and bias
    w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_io = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Forget gate: weights for input, weights for previous output, and bias
    w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_fo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Output gate: weights for input, weights for previous output, and bias
    w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_oo = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
    #Memory cell: weights for input, weights for previous output, and bias
    w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
    w_co = tf.Variable(tf.truncated_normal([hidden_nodes, hidden_nodes], -0.1, 0.1))
    b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
    #LCTM Cell
    def lstm(i, o, state):
        input_gate = tf.sigmoid(tf.matmul(i, w_ii) + tf.matmul(o, w_io) + b_i)
        forget_gate = tf.sigmoid(tf.matmul(i, w_fi) + tf.matmul(o, w_fo) + b_f)
        output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
        memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
        state = forget_gate * state + input_gate * memory_cell
        output = output_gate * tf.tanh(state)
        return output, state
    
    ###########
    #Operation
    ###########
    #LSTM
    output = tf.zeros([batch_size, hidden_nodes])
    state = tf.zeros([batch_size, hidden_nodes])

    for i in range(len_per_section):
        output, state = lstm(data[:, i, :], output, state)
        if i == 0:
            outputs_all_i = output
            labels_all_i = data[:, i+1, :]
        elif i != len_per_section - 1:
            outputs_all_i = tf.concat([outputs_all_i, output],0)
            labels_all_i = tf.concat([labels_all_i, data[:, i+1, :]],0)
        else:
            outputs_all_i = tf.concat([outputs_all_i, output],0)
            labels_all_i = tf.concat([labels_all_i, labels],0)
        
    #Classifier
    w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([char_size]))
    logits = tf.matmul(outputs_all_i, w) + b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels= labels_all_i))

    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step=global_step)
    
    ###########
    #Test
    ###########
    test_data = tf.placeholder(tf.float32, shape=[1, char_size])
    test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
    test_state = tf.Variable(tf.zeros([1, hidden_nodes]))
    
    #Reset at the beginning of each test
    reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
                                test_state.assign(tf.zeros([1, hidden_nodes])))

    #LSTM
    test_output, test_state = lstm(test_data, test_output, test_state)
    test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)
    

In [10]:
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    offset = 0
    saver = tf.train.Saver()
    
    for step in range(max_steps):
        offset = offset % len(X)
        if offset <= (len(X) - batch_size):
            batch_data = X[offset: offset + batch_size]
            batch_labels = y[offset: offset + batch_size]
            offset += batch_size
        else:
            to_add = batch_size - (len(X) - offset)
            batch_data = np.concatenate((X[offset: len(X)], X[0: to_add]))
            batch_labels = np.concatenate((y[offset: len(X)], y[0: to_add]))
            offset = to_add
        _, training_loss = sess.run([optimizer, loss], feed_dict={data: batch_data, labels: batch_labels})
        
        if step % log_every == 0:
            print('training loss at step %d: %.2f (%s)' % (step, training_loss, datetime.datetime.now()))

            if step % test_every == 0:
                reset_test_state.run()
                test_generated = test_start
                
                for i in range(len(test_start) - 1):
                    test_X = np.zeros((1, char_size))
                    test_X[0, char2id[test_start[i]]] = 1.
                    _ = sess.run(test_prediction, feed_dict={test_data: test_X})
                
                test_X = np.zeros((1, char_size))
                test_X[0, char2id[test_start[-1]]] = 1.
                
                for i in range(500):
                    prediction = test_prediction.eval({test_data: test_X})[0]
                    next_char_one_hot = sample(prediction)
                    next_char = id2char[np.argmax(next_char_one_hot)]
                    test_generated += next_char
                    test_X = next_char_one_hot.reshape((1, char_size))
                    
                print('=' * 80)
                print(test_generated)
                print('=' * 80)
                
                saver.save(sess, checkpoint_directory + '/model', global_step=step)

training loss at step 0: 4.79 (2017-12-16 15:47:03.995474)
I am thinking that                 r                                                                          t                                 o   o                            o                                             a                                                                                d                                                 e                                    n                                                                                                   e                          
training loss at step 3000: 2.18 (2017-12-16 15:56:22.619931)
training loss at step 6000: 1.97 (2017-12-16 16:05:41.377636)
I am thinking thate fie 12 thapain pes thangs” bord bof
dowa of he fin ma  bis!_ ha tere Sbl toupe | athe
casouretopwhe etan tyed. fome  bowe a agl-330 B ses a bof onowests bingoronce thicl e athoud pkesomep at tin gor f t kstwe aten m kinewovery f thera wafl pe sorigll ickerluaware H
doupa howith

training loss at step 63000: 1.49 (2017-12-16 19:06:36.058584)
training loss at step 66000: 1.51 (2017-12-16 19:15:52.737988)
I am thinking that he._, wal,” om ngh oft d, owiceas d if t, my tesonsulllinsers at chite malen k, wore bun thitheandn Peerad tn.
kiny
wst Thoritowof t
ss occheryong bed meldir herely d, juron m 6 ces f and kenitouvithe
carealor trt f “byee 22_d ig hind, ofl
ldomy tinf in
thinerryo n. omee in
h, so dlio coven, tif tethin l t hespt in-m ilk t
for, wienevin “adun ce hoprime ty  toray ad gees ofone br
t
bbe anuy t tot etote whare t o s, ave oure tharnar an, s “d it, olais. an t e tomyond’eansotingxpentume din t, s g
training loss at step 69000: 1.71 (2017-12-16 19:25:26.638504)
training loss at step 72000: 1.55 (2017-12-16 19:34:56.879383)
I am thinking thatl ndis the d, s nd stothe id s tifume ser orsomat „ Houc. as thenxpewan m a torned the abereg.”
Al then.
we tce
es hist, nve
th whind imamoremomanndon im” be iengKoleretencympe Hr
_t ube geno s chove bond wathem

In [11]:
test_start = 'I plan to make the world a better place '

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    model = tf.train.latest_checkpoint(checkpoint_directory)
    saver = tf.train.Saver()
    saver.restore(sess, model)

    reset_test_state.run()
    test_generated = test_start

    for i in range(len(test_start) - 1):
        test_X = np.zeros((1, char_size))
        test_X[0, char2id[test_start[i]]] = 1.
        _ = sess.run(test_prediction, feed_dict={test_data: test_X})

    test_X = np.zeros((1, char_size))
    test_X[0, char2id[test_start[-1]]] = 1.

    for i in range(500):
        prediction = test_prediction.eval({test_data: test_X})[0]
        next_char_one_hot = sample(prediction)
        next_char = id2char[np.argmax(next_char_one_hot)]
        test_generated += next_char
        test_X = next_char_one_hot.reshape((1, char_size))

    print(test_generated)
   

INFO:tensorflow:Restoring parameters from ckpt\model-78000
I plan to make the world a better place tr,
put, the tin. an acreveif wharer lad mienyofot, Mr se he farate ong
tht otaishaclll.  tursomalatese
s
 ay rto aur ather ofthe del pe onotit cagrarimenelin_ly ‘rin I Thare at chancatof om s n  wot Ja or m-w t msh findn tur d th Atht, baiad, ar-w olle oman feves gemainus: oug the, bit.’ t ame ts firceragstan, ain pe.
engleenare
m s an con nertong at st titin ound see whethila d w ce W8_llarurat it’s” os EBim ttre od
cersent ak.
wh thof als thatho wo unckig _sth trithiole pavivet s. merys is th
